In [1]:
!pip3 install torch==2.2.0 torchtext==0.17.0 GPUtil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
   

In [6]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torch.functional as F
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

import numpy,pandas
# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [7]:
with open("paul_graham_essays.txt", "r") as f:
    text=f.read()

print(len(text))
text[:200]

3023219


"  Anyone can see they're not the same by the number of people who are smart, but not very wise.  And yet intelligence and wisdom do seem related.  How?What is wisdom?  I'd say it's knowing what to do "

In [8]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|} ²àéö—≈⟨⟩
106


In [9]:
stoi={ch:i for i,ch in enumerate(chars)} # string to index
itos={i:ch for i,ch in enumerate(chars)}
def encode(s):
    return [stoi[ch] for ch in s]

def decode(n):
    return ''.join([itos[k] for k in n])
print(encode('kabir'))
print(decode([75, 65, 66, 73, 82]))

[75, 65, 66, 73, 82]
kabir


In [10]:
data=torch.tensor(encode(text[:1350000]),dtype=torch.long)
print(data.type,data.shape)

<built-in method type of Tensor object at 0x78788471c540> torch.Size([1350000])


In [11]:
ratio=int(0.9*len(data))
train_data=data[:ratio]
val_data=data[ratio:]
print(len(train_data),len(val_data))

1215000 135000


In [12]:
torch.manual_seed(42)
batch_size=64
block_size=257
def get_batch(split):
    data=train_data if split=="train" else val_data
    ix=torch.randint(len(data)-block_size,(batch_size, ))
    x=torch.stack([data[i:i+block_size] for i in ix])
    #y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x
xb=get_batch("train")
print(xb.shape)

torch.Size([64, 257])


In [13]:

device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:
device="cuda" if torch.cuda.is_available() else "cpu"
dropout=0.2
learning_rate=3e-4
n_head=6
n_layer=6
n_embd=384
print(device)

cuda


In [15]:
class GPT(nn.Module):

    def __init__(self,decoder,embed,generator):
        """
        GPT model architecture.
        """
        super(GPT,self).__init__()
        self.decoder=decoder
        self.embed=embed
        self.generator=generator

    def forward(self,x):
        #x-> [batch_size,seq_len]

        embedded=self.embed(x)
        return self.decoder(embedded)



In [16]:
class Generator(nn.Module):
    def __init__(self,d_model,vocab):
        super(Generator,self).__init__()
        self.proj=nn.Linear(in_features=d_model,out_features=vocab)

    def forward(self,x):
        #x -> [batch_size,seq_len,d_model]
        projection=self.proj(x) # [batch_size,seq_len,vocab]
        #pred=log_softmax(projection,dim=-1) # [batch_size,seq_len,vocab]
        return projection #since crossentropy expects raw logits
def clones(module,N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Decoder(nn.Module):
    def __init__(self,layer,N):
        super(Decoder,self).__init__()
        self.layers=clones(layer,N)
        self.norm=LayerNorm(layer.size)  # layer.size -> [d_model]

    def forward(self,x):
        for layer in self.layers:
            x=layer(x)
        # print("hello from decoder")


        return self.norm(x)

class LayerNorm(nn.Module):
    #https://tungmphung.com/wp-content/uploads/2020/01/Screenshot-from-2020-01-05-07-00-09.png
    def __init__(self,feature,eps=1e-6): #feature -> [d_model]
        super(LayerNorm,self).__init__()
        self.gamma=nn.Parameter(torch.ones(feature))
        self.beta=nn.Parameter(torch.zeros(feature))
        self.eps=eps

    def forward(self,x):
        mean=x.mean(-1,keepdim=True)
        std=x.std(-1,keepdim=True)

        return self.gamma*(x-mean)/(std+self.eps) + self.beta

class SubLayerConnection(nn.Module):
    """Residual connection followed by a layer norm """
    def __init__(self,size,dropout):
        super(SubLayerConnection,self).__init__()
        self.norm=LayerNorm(size)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,sublayer):
        # print("hello from sublayer")

        return x + self.dropout(sublayer(self.norm(x)))
class DecoderLayer(nn.Module):
    def __init__(self,size,self_attn,feed_forward,dropout):
        super(DecoderLayer,self).__init__()
        self.self_attn=self_attn
        self.feed_forward=feed_forward
        self.sublayer=clones(SubLayerConnection(size,dropout),2)
        self.size=size

    def forward(self,x):
        x=self.sublayer[0](x,lambda x: self.self_attn(x,x,x))
        return self.sublayer[1](x,self.feed_forward)


def attention(query,key,value,dropout=None):
    #query,key,value -> [batch_size,h,seq_len,d_k]
    d_k=query.size(-1)
    scores=torch.matmul(query,key.transpose(-2,-1))/math.sqrt(d_k) #[batch_size,h,seq_len,seq_len]
    # print("Query shape:", query.shape)
    # print("Key shape:", key.shape)
    # print("Value shape:", value.shape)


        #print('mask',mask.shape)
    seq_len=query.size(2)
    mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0).unsqueeze(0).to(device)
    scores=scores.masked_fill(mask==0,-1e9) # wherever mask==0, fill that up with -inf
        #print('scores',scores.shape[-1])
    p_attn=scores.softmax(dim=-1)
    if dropout is not None:
        p_attn=dropout(p_attn)
    return torch.matmul(p_attn,value),p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self,h,d_model,dropout=0.1):
        #h=12,N=12
        super(MultiHeadedAttention,self).__init__()
        assert d_model%h==0
        self.d_k=d_model//h
        self.h=h
        self.linears=clones(nn.Linear(d_model,d_model),4)
        self.attn=None
        self.dropout=nn.Dropout(p=dropout)

    def forward(self,query,key,value):
        #mask shape-> [batch_size,seq_len,seq_len]

        nbatches=query.size(0)

        query,key,value=[
            lin(x).view(nbatches,-1,self.h,self.d_k).transpose(1,2) for lin,x in zip(self.linears,(query,key,value))
        ]
        #print('mask fow',mask.shape)
        x,self_attn=attention(query,key,value,dropout=self.dropout)

        x=(
            x.transpose(1,2).contiguous().view(nbatches,-1,self.h*self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x) #WO * x

#3072
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,d_ff,dropout=0.1):
        #d_ff=3072
        super(PositionwiseFeedForward,self).__init__()
        self.w_1=nn.Linear(d_model,d_ff)
        self.w_2=nn.Linear(d_ff,d_model)
        self.dropout=nn.Dropout(p=dropout)
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )


    def forward(self,x):
        #print("hello from feeforward")
        return self.net(x)

class Embeddings(nn.Module):
    def __init__(self,d_model,vocab):
        super(Embeddings,self).__init__()
        self.lut=nn.Embedding(num_embeddings=vocab,embedding_dim=d_model)
        self.d_model=d_model

    def forward(self,x):
        return self.lut(x) * math.sqrt(self.d_model)
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout,max_len=5000):
        """
        Initializes the PositionalEncoding module.

        Args:
            d_model (int): The dimensionality of the input and output features.
            dropout (float): The dropout rate to be applied after the positional encoding.
            max_len (int, optional): The maximum length of the sequence to be encoded. Defaults to 5000.
        """
        super(PositionalEncoding,self).__init__()
        self.dropout=nn.Dropout(dropout)

        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # print('diveterm',div_term)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        #print("hello from posen")
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

def make_model(vocab,N=6,d_model=384,d_ff=1536,h=6,dropout=0.1):
    c=copy.deepcopy
    attn=MultiHeadedAttention(h,d_model)
    ff=PositionwiseFeedForward(d_model,d_ff,dropout)
    position=PositionalEncoding(d_model,dropout)
    model=GPT(
        Decoder(DecoderLayer(d_model,c(attn),c(ff),dropout),N),
        nn.Sequential(Embeddings(d_model,vocab),c(position)),
        Generator(d_model,vocab))

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [17]:
vocab_size

106

In [18]:
model=make_model(vocab=vocab_size,N=6,d_model=384,d_ff=1536,h=6,dropout=0.2).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion=nn.CrossEntropyLoss()
criterion


CrossEntropyLoss()

In [19]:
class Batch:
    def __init__(self, src, pad=0):
        self.src = src


        # Derive tgt and tgt_y from src
        self.tgt = src[:, :-1]  # Input to the decoder (shift right)
        self.tgt_y = src[:, 1:]  # Target for loss computation (shift left
        self.ntokens = (self.tgt_y != pad).data.sum()

In [20]:
criterion

CrossEntropyLoss()

In [21]:
class SimpleLossCompute:
    def __init__(self,generator,criterion,pad):
        self.generator=generator
        self.criterion=criterion

    def __call__(self,x,y,norm):
        logits=self.generator(x)
        loss=self.criterion(
            logits.contiguous().view(-1,logits.size(-1)),
            y.contiguous().view(-1)
        ) / norm

        return loss.data*norm,loss
loss_compute=SimpleLossCompute(model.generator,criterion,pad=0)

In [25]:
from tqdm import tqdm

max_iters = 9000
losses = []
save_interval = 1500

for iter in tqdm(range(max_iters), desc="Training Progress"):

    xb = get_batch("train")
    batch = Batch(xb)
    src = batch.tgt.to(device)
    tgt = batch.tgt_y.to(device)
    print('src',src.shape)
    out = model.forward(src)  # Shape: [batch_size, seq_len, d_model]

    logits = model.generator(out)  # Shape: [batch_size, seq_len, vocab_size]


    loss, loss_node = loss_compute(out, tgt, batch.ntokens.to(device))


    losses.append(loss.item())


    predicted = torch.argmax(logits, dim=-1)  # Predicted indices: [batch_size, seq_len]
    correct_predictions = (predicted == tgt).float().sum().item()
    total_characters = tgt.numel()  # Total characters in the target
    accuracy = (correct_predictions / total_characters) * 100  # Accuracy as a percentage

    tqdm.write(
        f"Iteration {iter + 1}/{max_iters}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%"
    )


    loss_node.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)


    if (iter + 1) % save_interval == 0:
        torch.save(model.state_dict(), f"model_weights_{iter+1}.pth")
        print(f"Saved model weights at iteration {iter + 1}")

Training Progress:   0%|          | 0/9000 [00:00<?, ?it/s]

src torch.Size([64, 256])
Iteration 1/9000, Loss: 5.2747, Accuracy: 17.95%


Training Progress:   0%|          | 2/9000 [00:00<50:56,  2.94it/s]

src torch.Size([64, 256])
Iteration 2/9000, Loss: 4.5428, Accuracy: 10.61%


Training Progress:   0%|          | 3/9000 [00:01<56:54,  2.63it/s]

...

src torch.Size([64, 256])
Iteration 8999/9000, Loss: 0.2263, Accuracy: 92.91%


Training Progress: 100%|█████████▉| 8999/9000 [1:07:18<00:00,  2.22it/s]

src torch.Size([64, 256])
Iteration 9000/9000, Loss: 0.2287, Accuracy: 92.91%


Training Progress: 100%|██████████| 9000/9000 [1:07:18<00:00,  2.23it/s]

Saved model weights at iteration 9000


In [30]:

def generate_next_words(model, input_seq, max_len=30, device='cpu'):

    model.eval()

    # Encode the input sequence and move to the device
    #input_ids = torch.LongTensor(tokenizer.encode(input_seq)).unsqueeze(0).to(device)
    #print(torch.LongTensor([word2idx[word] for word in input_seq.split() if word in word2idx]).unsqueeze(0).to(device))
    input_ids = torch.LongTensor([stoi[word] for word in input_seq if word in stoi]).unsqueeze(0).to(device)  # Shape: [1, seq_len]
    generated = input_ids  # Start the generated sequence with input

    for _ in range(max_len):
        # Generate predictions for the next token
        with torch.no_grad():
            seq_len = generated.size(1)  # Current sequence length

            output = model(generated)
            output=model.generator(output)

        next_token_logits = output[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        temperature = 0.8
        next_token_logits = next_token_logits / temperature
        probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)
        next_token_id = torch.multinomial(probabilities, num_samples=1)
        generated = torch.cat([generated, next_token_id], dim=1)

        if (generated >= vocab_size).any():
            #print("Generated sequence contains invalid token indices!")
            generated = generated.clamp(0, vocab_size - 1)  # Clamp to valid range
    indices = generated[0].tolist()  # Assuming we want to decode the first sequence in the batch

    # Map each index to its corresponding word
    words = [itos[idx] for idx in indices if idx in itos]

    # Join the words into a single string
    generated_text = ' '.join(words)
    return generated_text


In [38]:
def merge_characters_with_space(text):
    # Split text by spaces and filter out empty strings

    # Join the characters with a single space
    merged_text = ' '.join(text)
    return merged_text
text=generate_next_words(model, "recipies of wisdom", max_len=300,device=device)
text

'r e c i p i e s   o f   w i s d o m   c o m e s   a l s o   g o   t o   f i n d   o u t   s o m e t h i n g   b a d   p r o c e s s i n g   i n f l u e n c e d   i n   1 9 7 5 .   I f   t h e   c h o i c t i o n   i s   d y n a y   l i f e ,   t h e   a n s w e r   i s   y e s ,   t h e   f o r m e r   s h o u l d   f e e l   b e   o n   a   l o n g e r   t h a n   l i v e   t h e y   d i d .   B u t   h e   g o t   k i n d   o f     f i n d   o u t   t h e   a u d i e n c e ,   a i m   i n   f r i e l y   a p a r t h r t h r t t t t t t a r ;   2   2   2 . T h i e r e s t h i e v i e   a r k .   a t h i e   t h i s t h o i s  '

In [53]:
def clean(text):
  for i in text:
    for j in text:
      if i!=" " and j==" ":
        del j
  return text


str(clean(list(text)))

"['r', ' ', 'e', ' ', 'c', ' ', 'i', ' ', 'p', ' ', 'i', ' ', 'e', ' ', 's', ' ', ' ', ' ', 'o', ' ', 'f', ' ', ' ', ' ', 'w', ' ', 'i', ' ', 's', ' ', 'd', ' ', 'o', ' ', 'm', ' ', ' ', ' ', 'c', ' ', 'o', ' ', 'm', ' ', 'e', ' ', 's', ' ', ' ', ' ', 'a', ' ', 'l', ' ', 's', ' ', 'o', ' ', ' ', ' ', 'g', ' ', 'o', ' ', ' ', ' ', 't', ' ', 'o', ' ', ' ', ' ', 'f', ' ', 'i', ' ', 'n', ' ', 'd', ' ', ' ', ' ', 'o', ' ', 'u', ' ', 't', ' ', ' ', ' ', 's', ' ', 'o', ' ', 'm', ' ', 'e', ' ', 't', ' ', 'h', ' ', 'i', ' ', 'n', ' ', 'g', ' ', ' ', ' ', 'b', ' ', 'a', ' ', 'd', ' ', ' ', ' ', 'p', ' ', 'r', ' ', 'o', ' ', 'c', ' ', 'e', ' ', 's', ' ', 's', ' ', 'i', ' ', 'n', ' ', 'g', ' ', ' ', ' ', 'i', ' ', 'n', ' ', 'f', ' ', 'l', ' ', 'u', ' ', 'e', ' ', 'n', ' ', 'c', ' ', 'e', ' ', 'd', ' ', ' ', ' ', 'i', ' ', 'n', ' ', ' ', ' ', '1', ' ', '9', ' ', '7', ' ', '5', ' ', '.', ' ', ' ', ' ', 'I', ' ', 'f', ' ', ' ', ' ', 't', ' ', 'h', ' ', 'e', ' ', ' ', ' ', 'c', ' ', 'h', ' ', 'o', ' '

In [50]:
import re

def clean_text(text):
    # Replace double spaces with a single space
    for i in text:
      for j in text:
        if i!=" " and j==" ":
          del j
# Example usage
input_text = "r e c i p i e s   o f   w i s d o m   c o m e s   a l s o   g o   t o   f i n d   o u t ..."
output_text = clean_text(input_text)
print(output_text)

recipiesofwisdomcomesalsogotofindout...


In [41]:
def process_spaced_text(text):
    """
    Process text where characters are separated by single spaces:
    - Merge all single-character tokens into words

    Args:
        text (str): Input text with characters separated by spaces

    Returns:
        str: Processed text with merged characters
    """
    # Split the text into individual characters
    words = text.split()

    # Join the characters into words (without spaces)
    merged_text = ''.join(words)

    return merged_text

# Example usage
input_text = "r e c i p i e s   o f   w i s d o m"
processed_text = process_spaced_text(input_text)
print(processed_text)

recipiesofwisdom
